Následující report slouží k vygenerování seznamu sportovců, kteří se zúčastnili více než 6 dnů soutěží v daném období od 1.11.2023 do 30.10.2024.

Kde soutěže jsou filtrovány podle kategorie.

Dále se počítá počet dnů soutěží pomocí pole DatumOd a DatumDo v matrice, kde u některých vícetermínových turnajů je nutno datum ručně změnit, protože Od a Do jsou vždy jen datum prvního dne a posledního dne turnaje.


In [37]:
from src.matrikadb import connect_to_db, get_env_vars

credentials = get_env_vars()

conn = connect_to_db(**credentials)

Successfully connected to the database!


Exportujeme všechny účasti od 1.11.2023 do 30.10.2024. Náhled dat:

In [38]:
import pandas as pd

ucasti_sql =  """
SELECT 
    h.Legitimace,
    h.Prijmeni,
    h.Jmena,
    h.Rocnik,
    k.Nazev as KlubNazev,
    t.Nazev as TurnajNazev,
    t.DatumOd as TurnajOd,
    t.DatumDo as TurnajDo,
    kt.Nazev as KategorieTurnaje,
    CASE 
        WHEN p.Id IS NOT NULL THEN 'Ano'
        ELSE 'Ne'
    END as ClenPrispevek2024
FROM Hrac h
INNER JOIN Ucast u ON h.Legitimace = u.IdHrace
INNER JOIN Soutezici s ON s.Id = u.IdSouteziciho
INNER JOIN Turnaj t ON t.Id = s.IdTurnaje
LEFT JOIN Klub k ON k.Id = h.IdKlubu
LEFT JOIN KategorieTurnaje kt ON kt.Id = t.IdKategorie
LEFT JOIN Prispevek p ON p.IdHrace = h.Legitimace AND p.IdSezony = 2024
WHERE 
    t.DatumOd >= CONVERT(datetime, '2023-11-01', 120)
    AND t.DatumOd <= CONVERT(datetime, '2024-10-31', 120)
ORDER BY Prijmeni, Jmena
    """

df = pd.read_sql(ucasti_sql, conn)

# Vypočítáme počet dnů soutěží pro každou soutěž
df['DobaTurnaje'] = (pd.to_datetime(df['TurnajDo']) - pd.to_datetime(df['TurnajOd'])).dt.days

# Jednodenní soutěže jsou převedeny na 1 den
df.loc[df['DobaTurnaje'] == 0, 'DobaTurnaje'] = 1

df.head()

/var/folders/7n/9pr1nk311yb_s9yqf3yjh_500000gp/T/ipykernel_40213/1867312129.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(ucasti_sql, conn)


,Legitimace,Prijmeni,Jmena,Rocnik,KlubNazev,TurnajNazev,TurnajOd,TurnajDo,KategorieTurnaje,ClenPrispevek2024,DobaTurnaje
0,3728,Adamczak,Jakub,2009,Zahraniční,5th Central European Championship - Kids,2023-11-17,2023-11-17,Juniorské MČR,Ne,1
1,3728,Adamczak,Jakub,2009,Zahraniční,Funbridge (16. - 18. 11. 2023),2023-11-16,2023-11-18,Funbridge,Ne,2
2,3728,Adamczak,Jakub,2009,Zahraniční,Funbridge (11. - 13. 7. 2024),2024-07-11,2024-07-13,Funbridge,Ne,2
3,3723,Adamczak,Michal,2012,Zahraniční,5th Central European Championship - Kids,2023-11-17,2023-11-17,Juniorské MČR,Ne,1
4,2949,Adamová,Marie,1954,BK Máj České Budějovice,Klubové body České Budějovice leden 2024,2024-01-04,2024-01-11,Klubové a jiné podobné turnaje,Ano,7


## Kritérium zaplaceného členství

Vyfiltrujeme sportovce, kteří mají prispevek za rok 2024

In [39]:
df = df[df['ClenPrispevek2024'] == 'Ano']

print(f"Počet účastí s prispevekem za rok 2024: {len(df)}")

Počet účastí s prispevekem za rok 2024: 4594


## Kritérium svazových soutěží

Podívejme se na kategorie turnajů:

In [40]:
print("Kategorie:\n")

print(*sorted(df['KategorieTurnaje'].fillna('Neznámá').unique().tolist()), sep='\n' )


Kategorie:

1. liga
2. liga
3. liga
CBT
Funbridge
Juniorské MČR
Klubové a jiné podobné turnaje
MČR MIX
MČR TOP
Neznámá
Skupinovky, Malé ceny, Pražská liga
Slavonice
Vedlejší páráky VC/CBT, Hlavní párák Slavonice
Vedlejší týmy CBT
Velké ceny


Z těchto kategorií chceme vyfiltrovat: Funbridge, Klubové a jiné podobné turnaje

Dále je zde pár turnajů s neznámou kategorií, podívejme se na ně:

In [41]:

display( df[df['KategorieTurnaje'].isna()][['TurnajNazev', 'TurnajOd', 'TurnajDo']])

,TurnajNazev,TurnajOd,TurnajDo
50,Účastnické bonusové body - CBT,2024-02-17,2024-10-19
51,Účastnické bonusové body - MČR,2024-10-05,2024-12-07
55,Deštné,2024-02-10,2024-02-16
79,Účastnické bonusové body - CBT,2024-02-17,2024-10-19
82,Deštné,2024-02-10,2024-02-16
...,...,...,...
5846,Účastnické bonusové body - CBT,2024-02-17,2024-10-19
5847,Účastnické bonusové body - MČR,2024-10-05,2024-12-07
5892,Klubové body Havířov květen 2024,2024-05-07,2024-05-28
5944,Účastnické bonusové body - CBT,2024-02-17,2024-10-19


Vidíme, že jde o souhrnné turnaje, které nechceme počítat, tedy je také vyfiltrujeme:

In [42]:
print(f"Počet řádků před filtrováním: {len(df)}")
df = df[~df['KategorieTurnaje'].isin(['Funbridge', 'Klubové a jiné podobné turnaje', None])]
print(f"Počet řádků po filtrování: {len(df)}")
print("\nZbývající kategorie:")
print(*sorted(df['KategorieTurnaje'].dropna().unique().tolist()), sep='\n')


Počet řádků před filtrováním: 4594
Počet řádků po filtrování: 1718

Zbývající kategorie:
1. liga
2. liga
3. liga
CBT
Juniorské MČR
MČR MIX
MČR TOP
Skupinovky, Malé ceny, Pražská liga
Slavonice
Vedlejší páráky VC/CBT, Hlavní párák Slavonice
Vedlejší týmy CBT
Velké ceny


## Výpočet počtu dnů soutěží

Nyní se zaměřmě na výpočet počtu dnů soutěží.

Podívejme se na soutěže a jejich počet dnů, konkrétně na soutěže, které trvají více než 2 dny:


In [43]:

df_tournaments = df[['TurnajNazev', 'TurnajOd', 'TurnajDo', 'DobaTurnaje']].drop_duplicates()

df_long_tournaments =  df_tournaments[df_tournaments['DobaTurnaje'] > 2]

df_long_tournaments.sort_values(by='TurnajNazev', ascending=False)


,TurnajNazev,TurnajOd,TurnajDo,DobaTurnaje
275,Švýcarská skupinovka podzim 2024,2024-09-18,2024-11-13,56
193,Skupinová B Podzim 2024,2024-09-25,2025-01-29,126
52,Skupinová A Podzim 2024,2024-09-23,2024-11-18,56
16,Skupinovka České Budějovice - Podzim 2024,2024-09-19,2025-02-13,147
8,Skupinovka České Budějovice - Jaro 2024,2024-02-22,2024-06-13,112
254,Pražská švýcarská skupinovka jaro 2024,2024-02-08,2024-06-13,126
35,Pražská skupinová A zima 2023,2023-11-20,2024-01-29,70
41,Pražská skupinová A léto 2024,2024-05-01,2024-06-24,54
42,Pražská skupinovka A Jaro 2024,2024-02-15,2024-04-30,75
43,Pražská liga jaro 2024,2024-02-12,2024-06-03,112


### Doplnění počtu dnů soutěží pro dlouhodobé turnaje

U dlouhodobých turnajů je nutné ručně změnit počet dnů soutěží, protože Od a Do jsou vždy jen datum prvního dne a posledního dne turnaje.

In [44]:



long_tournament_mapping = {
 'Skupinovka České Budějovice - Jaro 2024': 6,
 'Skupinovka České Budějovice - Podzim 2024': 6,
 'Pražská skupinová A zima 2023': 5,
 '1. Liga 2024': 8,
 'Skupinová A - Jaro 2024': 5,
 'Pražská skupinová A léto 2024': 5,
 'Pražská skupinovka A Jaro 2024': 5,
 'Pražská liga jaro 2024': 10,
 "Skupinová A Podzim 2024": 5,
 '2. Liga 2024': 8,
 'Skupinová B Podzim 2024': 9,
 'Pražská Skupinová B Jaro 2024': 9,
 'Pražská švýcarská skupinovka jaro 2024': 5,
 "Švýcarská skupinovka podzim 2024": 5,
 'Brno skupinovka 09/24 - 01/25': 6,
 'Brno - Skupinovka': 6,
}

# Upravíme počet dnů soutěží pro dlouhodobé turnaje
for tournament, days in long_tournament_mapping.items():
    df.loc[df['TurnajNazev'] == tournament, 'DobaTurnaje'] = days

# Zobrazime mapovani
for tournament, days in long_tournament_mapping.items():
    print(f"{tournament}".ljust(45), f"{days} dní")



Skupinovka České Budějovice - Jaro 2024       6 dní
Skupinovka České Budějovice - Podzim 2024     6 dní
Pražská skupinová A zima 2023                 5 dní
1. Liga 2024                                  8 dní
Skupinová A - Jaro 2024                       5 dní
Pražská skupinová A léto 2024                 5 dní
Pražská skupinovka A Jaro 2024                5 dní
Pražská liga jaro 2024                        10 dní
Skupinová A Podzim 2024                       5 dní
2. Liga 2024                                  8 dní
Skupinová B Podzim 2024                       9 dní
Pražská Skupinová B Jaro 2024                 9 dní
Pražská švýcarská skupinovka jaro 2024        5 dní
Švýcarská skupinovka podzim 2024              5 dní
Brno skupinovka 09/24 - 01/25                 6 dní
Brno - Skupinovka                             6 dní


## Export seznamu sportovců

Nyní můžeme vygenerovat seznam sportovců, kteří se zúčastnili více než 6 dnů soutěží v daném období od 1.11.2023 do 30.10.2024.

[Odkaz na stažení seznamu sportovců](sportovci_nad_6_dni.csv)  

[Odkaz na stažení seznamu sportovců s rokem narození mezi 1975 a 2001](sportovci_nad_6_dni_filtered.csv)

In [45]:
# Seskupíme data podle hráče a sečteme dny turnajů
import os


sportovci_dny = df.groupby(['Legitimace', 'Prijmeni', 'Jmena', 'Rocnik', 'KlubNazev'])['DobaTurnaje'].sum().reset_index().rename(columns={'DobaTurnaje': 'Pocet dni'})

# Vyfiltrujeme hráče s více než 6 dny soutěží
sportovci_nad_6_dni = sportovci_dny[sportovci_dny['Pocet dni'] >= 6].sort_values('Pocet dni', ascending=False)
sportovic_nad_6_dni_filtered = sportovci_nad_6_dni[(sportovci_nad_6_dni['Rocnik'] >= 1975) & (sportovci_nad_6_dni['Rocnik'] <= 2001)]

# Zobrazíme výsledky
print(f"Počet sportovců s více než 6 dny soutěží: {len(sportovci_nad_6_dni)}")
print(f"Počet sportovců s více než 6 dny soutěží a rokem narození mezi 1975 a 2001: {len(sportovic_nad_6_dni_filtered)}")
print("\nSeznam sportovců:")
os.makedirs('out', exist_ok=True)
sportovci_nad_6_dni.to_csv('out/sportovci_nad_6_dni.csv', index=False)
sportovic_nad_6_dni_filtered.to_csv('out/sportovci_nad_6_dni_filtered.csv', index=False)


sportovci_nad_6_dni


Počet sportovců s více než 6 dny soutěží: 208
Počet sportovců s více než 6 dny soutěží a rokem narození mezi 1975 a 2001: 62

Seznam sportovců:


,Legitimace,Prijmeni,Jmena,Rocnik,KlubNazev,Pocet dni
24,341,Svoboda,Otakar,1953,BK Praha,87
72,1761,Spálovský,Luboš,1961,BK Praha,83
57,1188,Nulíček,Vladimír,1959,BK Praha,80
73,1782,Raboch,Pavel,1973,BK Praha,77
8,187,Hnátová,Daniela,1969,BK Praha,76
...,...,...,...,...,...,...
213,3642,Barb,Simon,1958,Zahraniční,8
92,2146,Bušek,Vladimír,1982,BK Uherské Hradiště,8
127,2684,Bielecki,Piotr,1954,BS Havířov,8
173,3056,Čech,Martin,1970,BK Brno,7


In [46]:

print("Seznam sportovců s více než 6 dny soutěží a rokem narození mezi 1975 a 2001:")
sportovic_nad_6_dni_filtered

Seznam sportovců s více než 6 dny soutěží a rokem narození mezi 1975 a 2001:


,Legitimace,Prijmeni,Jmena,Rocnik,KlubNazev,Pocet dni
207,3559,Franz,Vojtěch,2000,BK Praha,61
158,2909,Volhejn,Petr,2001,BK Trutnov z.s.,57
78,1868,Vozábal,David,1978,BS Havířov,56
134,2733,Kohutová,Lucie,1997,BS Havířov,52
89,2095,Bahník,Ondřej,1985,BK Chaos Bridge,51
...,...,...,...,...,...,...
104,2304,Medlín,Jiří,1977,BK Praha,8
115,2462,Teichmann,Lukáš,1990,BS Havířov,8
129,2701,Bouřa,Patrik,1995,BK Uherské Hradiště,8
124,2672,Kolek,Lukáš,1995,BS Havířov,8
